In [104]:
import os
import re
import numpy as np
import pandas as pd

In [105]:
train_test_path = os.path.dirname(os.path.abspath(""))
os.listdir(train_test_path)

['ML', 'test.csv', 'train.csv']

In [106]:
train_csv_parh = os.path.join(train_test_path, "train.csv")
test_csv_parh = os.path.join(train_test_path, "test.csv")

In [267]:
df = pd.read_csv(train_csv_parh)
df = df.set_index("PassengerId") # set this col as index
df_test = pd.read_csv(test_csv_parh)
df

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S


In [268]:
# most missing values on Cabin column which is categorical type of data (better to drop it)
# missing values on age column will be restored by median/mean preprocessing
df.isnull().sum()

Survived      0
Pclass        0
Name          0
Sex           0
Age         177
SibSp         0
Parch         0
Ticket        0
Fare          0
Cabin       687
Embarked      2
dtype: int64

In [269]:
df["Cabin"].value_counts()

B96 B98        4
G6             4
C23 C25 C27    4
C22 C26        3
F33            3
              ..
E34            1
C7             1
C54            1
E36            1
C148           1
Name: Cabin, Length: 147, dtype: int64

### feature engineering
* add new col FamilySize that is sum of SibSp and Parch
* drop numerical values from Cabin col
* extract titles from Name col
* imputate missing data on Age col

In [270]:
# FamilySize
df["FamilySize"] = df["SibSp"] + df["Parch"]
#drop SibSp and Parch cols
df = df.drop(columns=["SibSp", "Parch"])
df

,Survived,Pclass,Name,Sex,Age,Ticket,Fare,Cabin,Embarked,FamilySize
PassengerId,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,A/5 21171,7.2500,NaN,S,1
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,PC 17599,71.2833,C85,C,1
3,1,3,"Heikkinen, Miss. Laina",female,26.0,STON/O2. 3101282,7.9250,NaN,S,0
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,113803,53.1000,C123,S,1
5,0,3,"Allen, Mr. William Henry",male,35.0,373450,8.0500,NaN,S,0
...,...,...,...,...,...,...,...,...,...,...
887,0,2,"Montvila, Rev. Juozas",male,27.0,211536,13.0000,NaN,S,0
888,1,1,"Graham, Miss. Margaret Edith",female,19.0,112053,30.0000,B42,S,0
889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,W./C. 6607,23.4500,NaN,S,3


In [271]:
# observation of Cabin column
cabins = df["Cabin"].dropna()
cabins.head(40)

PassengerId
2              C85
4             C123
7              E46
11              G6
12            C103
22             D56
24              A6
28     C23 C25 C27
32             B78
53             D33
55             B30
56             C52
62             B28
63             C83
67             F33
76           F G73
89     C23 C25 C27
93             E31
97              A5
98         D10 D12
103            D26
111           C110
119        B58 B60
124           E101
125            D26
129          F E69
137            D47
138           C123
140            B86
149             F2
152             C2
167            E33
171            B19
175             A7
178            C49
184             F4
186            A32
194             F2
195             B4
196            B80
Name: Cabin, dtype: object

In [272]:
# dropping numerical data from Cabin col
pattern = re.compile(r'[a-zA-Z]+')
df_iter_copy = df.copy()
for n,i in enumerate(df_iter_copy["Cabin"], start=1):
    matches = []
    if type(i) == str:
        i = "".join(i.split(" ")).strip()
        matches = pattern.findall(i)
        matches = list(set(matches))[0] # remove duplicates
        if len(matches) > 1:
            matches = "Not sure"
        df.loc[n, "Cabin"] = matches
#         print(df["Cabin"].iloc[n])
    continue
df["Cabin"]

PassengerId
1      NaN
2        C
3      NaN
4        C
5      NaN
      ... 
887    NaN
888      B
889    NaN
890      C
891    NaN
Name: Cabin, Length: 891, dtype: object

In [273]:
#* fill NaN by new category "Not Sure"
df["Cabin"] = df["Cabin"].fillna("Not Sure")
df["Cabin"]

PassengerId
1      Not Sure
2             C
3      Not Sure
4             C
5      Not Sure
         ...   
887    Not Sure
888           B
889    Not Sure
890           C
891    Not Sure
Name: Cabin, Length: 891, dtype: object

In [274]:
df.isnull().sum()

Survived        0
Pclass          0
Name            0
Sex             0
Age           177
Ticket          0
Fare            0
Cabin           0
Embarked        2
FamilySize      0
dtype: int64

In [275]:
# extract titles from Name col
all_titles = ["Dr." , "Mr.", "Mrs.", "Ms.", "Miss.", 
              "Master.", "Rev.", "Mlle.", "Major."]
positions, titles = [], []
for n, i in enumerate(df["Name"].str.split(), start=1):
        for title in all_titles:
            if title in i:
                positions.append(n)
                titles.append(title)

In [276]:
df["Name"].value_counts()

Braund, Mr. Owen Harris                     1
Boulos, Mr. Hanna                           1
Frolicher-Stehli, Mr. Maxmillian            1
Gilinski, Mr. Eliezer                       1
Murdlin, Mr. Joseph                         1
                                           ..
Kelly, Miss. Anna Katherine "Annie Kate"    1
McCoy, Mr. Bernard                          1
Johnson, Mr. William Cahoone Jr             1
Keane, Miss. Nora A                         1
Dooley, Mr. Patrick                         1
Name: Name, Length: 891, dtype: int64

In [277]:
# create new col and populate by positons and titles
df.loc[positions, "Title"] = titles
df["Title"]

PassengerId
1        Mr.
2       Mrs.
3      Miss.
4       Mrs.
5        Mr.
       ...  
887     Rev.
888    Miss.
889    Miss.
890      Mr.
891      Mr.
Name: Title, Length: 891, dtype: object

In [278]:
# drop rows in which title is not included in all_titles list
df = df[df["Title"].isin(all_titles) == True]

In [280]:
df

,Survived,Pclass,Name,Sex,Age,Ticket,Fare,Cabin,Embarked,FamilySize,Title
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,A/5 21171,7.2500,Not Sure,S,1,Mr.
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,PC 17599,71.2833,C,C,1,Mrs.
3,1,3,"Heikkinen, Miss. Laina",female,26.0,STON/O2. 3101282,7.9250,Not Sure,S,0,Miss.
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,113803,53.1000,C,S,1,Mrs.
5,0,3,"Allen, Mr. William Henry",male,35.0,373450,8.0500,Not Sure,S,0,Mr.
...,...,...,...,...,...,...,...,...,...,...,...
887,0,2,"Montvila, Rev. Juozas",male,27.0,211536,13.0000,Not Sure,S,0,Rev.
888,1,1,"Graham, Miss. Margaret Edith",female,19.0,112053,30.0000,B,S,0,Miss.
889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,W./C. 6607,23.4500,Not Sure,S,3,Miss.


In [289]:
# Imputation of Age missing values
from sklearn.impute import SimpleImputer

median_imputer = SimpleImputer(strategy="median")
median_imputer.fit(df["Age"].to_numpy().reshape(-1,1))
median_imputer.statistics_

array([28.])

In [302]:
#revise it!
age_median_imputed = median_imputer.transform(df["Age"].to_numpy().reshape(-1,1))
age_median_imputed.reshape(-1)
print(df["Age"].shape, age_median_imputed.shape)
df = df.loc[:, "Age"] = age_median_imputed
df.isnull().sum()

(882,) (882, 1)


AttributeError: 'numpy.ndarray' object has no attribute 'loc'